## TMA4215 - Project #1 ## 
*27.09.2024*

*Candidate ID*

In this project I have used numerical methods for Fourier series based signal processing.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit, njit

#### Task 1: The (Discrete) Fourier Transform

**a)** Considering the functions $e^{2\pi ikx}$, $k \in \mathbb{Z}$, $x \in \mathbb{T}$. I am going to below prove that for any $k, h \in \mathbb{Z}$ we have

$$
\left\langle e^{2\pi i k \cdot}, e^{2\pi i h \cdot} \right\rangle = 
\begin{cases} 
1 & \text{if } k = h \\
0 & \text{else}
\end{cases}
$$


The inner product of two functions $f(x)$ and $g(x)$ on the interval $[0, 1)$ is defined as:

$$
\langle f, g \rangle = \int_0^1 f(x) \overline{g(x)} \, dx
$$

For our case, $f(x) = e^{2\pi i k x}$ and $g(x) = e^{2\pi i h x}$. The complex conjugate of $g(x)$ is:

$$
\overline{g(x)} = e^{-2\pi i h x}
$$

Thus, the inner product becomes:

$$
\langle e^{2\pi i k x}, e^{2\pi i h x} \rangle = \int_0^1 e^{2\pi i k x} \cdot e^{-2\pi i h x} \, dx = \int_0^1 e^{2\pi i (k - h) x} \, dx
$$

We have two cases:

1. **$k = h$:**

$$
\int_0^1 e^{2\pi i (k - h) x} \, dx = \int_0^1 e^{0} \, dx = \int_0^1 1 \, dx = 1
$$

So, the inner product is 1 when $k = h$.

2. **$k \neq h$:**

In this case, let $m = k - h \neq 0$. We need to evaluate:

$$
\int_0^1 e^{2\pi i m x} \, dx
$$

The antiderivative of $e^{2\pi i m x}$ is:

$$
\int e^{2\pi i m x} \, dx = \frac{e^{2\pi i m x}}{2\pi i m}
$$

Evaluating this from $0$ to $1$ gives:

$$
\left[ \frac{e^{2\pi i m x}}{2\pi i m} \right]_0^1 = \frac{e^{2\pi i m \cdot 1} - e^{2\pi i m \cdot 0}}{2\pi i m} = \frac{e^{2\pi i m} - 1}{2\pi i m}
$$

Since $e^{2\pi i m} = 1$ for any integer $m$, we have:

$$
\frac{1 - 1}{2\pi i m} = 0
$$

So, the inner product is $0$ when $k \neq h$.

$\square$


**b)**

#### c)

#### d)

#### e)

#### f)

#### g)

#### h)

#### i)

#### j)

#### Task 2: Signal Processing

#### a)

#### b)

#### c)

#### d)

#### e)

#### Task 3: Sound Processing

#### a)

#### b)

#### c)

#### d)

#### e)

#### f)

#### g)